### Initialization including packages and data

#### Packages

In [9]:
import pandas as pd
import matplotlib.pyplot as plt
import gurobipy as gp
from gurobipy import GRB

#### Reading data into dataframes

In [10]:
df_genTech = pd.read_csv(r"data\TechnicalDataofGeneratingUnits-Table1.csv")

df_gen = pd.read_csv(r"data\CostsandInitialStateofGeneratingUnits-Table2.csv")

df_load = pd.read_csv(r"data\LoadProfile-Table3.csv")

df_demand = pd.read_csv(r"data\NodeLocationandDistributionoftheTotalSystemDemand-Table4.csv")
df_demand["Percent"] = df_demand["Percent"]/100

df_renew = pd.read_csv(r"data\scen_zone1.out", nrows = 24)

df_utility = pd.read_csv(r"data\demand_utility_prices_24h.csv")


### Copper-Plate, Single Hour

#### System model

In [11]:
# Initialize Gurobi model
model = gp.Model('Copperplate')

#Hours in a day:
time = 13

#Constants
cap_re = 200 #Capacity on the renewable generators
C_ren = 0 #Cost of renewable generation

N_d = len(df_demand) #Total number of loads
N_gen = len(df_genTech) # Total number of generators
N_ren = 6 # Total number of renewable generators

# Variables
Pgen = model.addVars(N_gen, vtype=gp.GRB.CONTINUOUS, name="P_gen")
Pd = model.addVars(N_d, vtype=gp.GRB.CONTINUOUS, name="P_demand")
Pw = model.addVars(N_ren, vtype = gp.GRB.CONTINUOUS, name="P_wind")


# Demand Capacity Constraints
model.addConstrs((Pd[d] <= df_demand.loc[d,"Percent"] * df_load.loc[time,"Demand"]
                for d in range(N_d)), name ="Demand Max Capacity")

model.addConstrs((Pd[d] >= 0
                for d in range(N_d)), name ="Demand Min Capacity")

# Generator Capacity Constraints
model.addConstrs((Pgen[gen] <= df_genTech.loc[gen,"Pmax"]
                for gen in range(N_gen)), name ="Generator Max Capacity")

model.addConstrs((Pgen[gen] >= 0
                for gen in range(N_gen)), name ="Generator Min Capacity")
#model.addConstrs((Pgen[gen] >= df_genTech.loc[gen,"Pmin"]
#                for gen in range(N_gen)), name ="Generator Min Capacity")


# Renewable Capacity Constraints
model.addConstrs((Pw[ren] <= df_renew.iloc[time,1 + ren] * cap_re
                for ren in range(N_ren)), name ="Renewable Max Capacity")

model.addConstrs((Pw[ren] >= 0
                for ren in range(N_ren)), name ="Renewable Min Capacity")


# Power balance constraint
PB_single = model.addConstr(sum(Pgen[gen] for gen in range(N_gen))
              + sum(Pw[ren] for ren in range(N_ren)) 
              - sum(Pd[d] for d in range(N_d)) == 0, name="Power Balance")


model.setObjective(sum(Pd[d] * df_utility.iloc[time,d+1] for d in range(N_d))
    - sum(Pgen[gen] * df_gen.loc[gen,"Ci"] for gen in range(N_gen))
    - sum(Pw[ren] * C_ren for ren in range(N_ren)),
    sense=GRB.MAXIMIZE
)
model.update()

# Solve the optimization problem
model.optimize()

Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (win64 - Windows 11+.0 (26200.2))

CPU model: AMD Ryzen 5 6600U with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 71 rows, 35 columns and 105 nonzeros
Model fingerprint: 0xc88e2e4d
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [5e+00, 4e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [6e+01, 6e+02]
Presolve removed 70 rows and 9 columns
Presolve time: 0.01s
Presolved: 1 rows, 26 columns, 26 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    9.9505759e+04   1.762666e+02   0.000000e+00      0s
       1    8.8491159e+04   0.000000e+00   0.000000e+00      0s

Solved in 1 iterations and 0.01 seconds (0.00 work units)
Optimal objective  8.849115902e+04


#### Data processing and analysis

In [12]:
clear_price_single = []
# Clearing price for each hour:
clear_price_single.append(PB_single.Pi)

### Copper-Plate, Multiple Hour (including storage unit)

#### System model

##### Copper-Plate, Multiple Hour (excluding storage unit)

In [ ]:
# Initialize Gurobi model
model = gp.Model('Copperplate-multi-nostorage')

#Hours in a day:
T = len(df_load)

#Constants
cap_re = 200 #Capacity on the renewable generators

N_d = len(df_demand) #Total number of loads
N_gen = len(df_genTech) # Total number of generators
N_ren = 6 # Total number of renewable generators

# Variables
Pgen = model.addVars(T,N_gen, vtype=gp.GRB.CONTINUOUS, name="P_gen")
Pd = model.addVars(T,N_d, vtype=gp.GRB.CONTINUOUS, name="P_demand")
Pw = model.addVars(T,N_ren, vtype = gp.GRB.CONTINUOUS, name="P_wind")

# Demand Capacity Constraints
model.addConstrs((Pd[t,d] <= df_demand.loc[d,"Percent"] * df_load.loc[t,"Demand"]
                for d in range(N_d)
                for t in range(T)), name ="Demand Max Capacity")

model.addConstrs((Pd[t,d] >= 0
                for d in range(N_d)
                for t in range(T)), name ="Demand Min Capacity")


# Generator Capacity Constraints
model.addConstrs((Pgen[t,gen] <= df_genTech.loc[gen,"Pmax"]
                for gen in range(N_gen)
                for t in range(T)), name ="Generator Max Capacity")

#model.addConstrs((Pgen[t,gen] >= df_genTech.loc[gen,"Pmin"] DO WE NEED MINIMUM DISPATCH OF GENERATORS?
#                for gen in range(N_gen)
#                for t in range(T)), name ="Generator Min Capacity")
model.addConstrs((Pgen[t,gen] >= 0
                for gen in range(N_gen)
                for t in range(T)), name ="Generator Min Capacity")


# Renewable Capacity Constraints
model.addConstrs((Pw[t,ren] <= df_renew.iloc[t,1 + ren] * cap_re
                for ren in range(N_ren)
                for t in range(T)), name ="Renewable Max Capacity")

model.addConstrs((Pw[t,ren] >= 0
                for ren in range(N_ren)
                for t in range(T)), name ="Renewable Min Capacity")


# Power balance constraint
# Create power balance constraint for each t in T
# so that we dont allow overproduction in one hour to be used in another hour
power_balance = model.addConstrs((
                sum(Pgen[t,gen] for gen in range(N_gen))
              + sum(Pw[t,ren] for ren in range(N_ren))
              == 
              sum(Pd[t,d] for d in range(N_d)) for t in range(T)
             ), name="Power_Balance")

# Storage is not included in the objective function, described in the Note 2 of the assignment description
# It can be utilized, but does neither bid or offer on energy market

# Objective function
model.setObjective(sum(
                 sum(Pd[t,d] * df_utility.iloc[t,d] for d in range(N_d))
                 - sum(Pgen[t,gen] * df_gen.loc[gen,"Ci"] for gen in range(N_gen))
                 - sum(Pw[t,ren] * 0 for ren in range(N_ren)) for t in range(T)), sense=GRB.MAXIMIZE)

model.update()

# Solve the optimization problem
model.optimize()




Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (win64 - Windows 11+.0 (26200.2))

CPU model: AMD Ryzen 5 6600U with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 1704 rows, 840 columns and 2520 nonzeros
Model fingerprint: 0x732fed53
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 5e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e+01, 6e+02]
Presolve removed 1680 rows and 217 columns
Presolve time: 0.01s
Presolved: 24 rows, 623 columns, 623 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    2.0325013e+06   3.239716e+03   0.000000e+00      0s
      24    1.8455392e+06   0.000000e+00   0.000000e+00      0s

Solved in 24 iterations and 0.01 seconds (0.00 work units)
Optimal objective  1.845539231e+06


##### Copper-Plate, Multiple Hour (including storage unit)

In [ ]:
# Initialize Gurobi model
model = gp.Model('Copperplate-multi-storage')


#Hours in a day:
T = len(df_load)

#Constants
cap_re = 200 #Capacity on the renewable generators

N_d = len(df_demand) #Total number of loads
N_gen = len(df_genTech) # Total number of generators
N_ren = 6 # Total number of renewable generators

#Constants for storage unit
E = 100 #MWh
P_ch = 50 #MW
P_dis = 50 #MW
eta_ch = 0.9 
eta_dis = 0.95

# Variables
Pgen = model.addVars(T,N_gen, vtype=gp.GRB.CONTINUOUS, name="P_gen")
Pd = model.addVars(T,N_d, vtype=gp.GRB.CONTINUOUS, name="P_demand")
Pw = model.addVars(T,N_ren, vtype = gp.GRB.CONTINUOUS, name="P_wind")

Pch = model.addVars(T, vtype=gp.GRB.CONTINUOUS, name="P_charge")
Pdis = model.addVars(T, vtype=gp.GRB.CONTINUOUS, name="P_discharge")
e = model.addVars(T, vtype=gp.GRB.CONTINUOUS, name="Storage")


# Demand Capacity Constraints
model.addConstrs((Pd[t,d] <= df_demand.loc[d,"Percent"] * df_load.loc[t,"Demand"]
                for d in range(N_d)
                for t in range(T)), name ="Demand Max Capacity")

model.addConstrs((Pd[t,d] >= 0
                for d in range(N_d)
                for t in range(T)), name ="Demand Min Capacity")


# Generator Capacity Constraints
model.addConstrs((Pgen[t,gen] <= df_genTech.loc[gen,"Pmax"]
                for gen in range(N_gen)
                for t in range(T)), name ="Generator Max Capacity")

#model.addConstrs((Pgen[t,gen] >= df_genTech.loc[gen,"Pmin"] DO WE NEED MINIMUM DISPATCH OF GENERATORS?
#                for gen in range(N_gen)
#                for t in range(T)), name ="Generator Min Capacity")
model.addConstrs((Pgen[t,gen] >= 0
                for gen in range(N_gen)
                for t in range(T)), name ="Generator Min Capacity")


# Renewable Capacity Constraints
model.addConstrs((Pw[t,ren] <= df_renew.iloc[t,1 + ren] * cap_re
                for ren in range(N_ren)
                for t in range(T)), name ="Renewable Max Capacity")

model.addConstrs((Pw[t,ren] >= 0
                for ren in range(N_ren)
                for t in range(T)), name ="Renewable Min Capacity")


# Power balance constraint
# Create power balance constraint for each t in T
# so that we dont allow overproduction in one hour to be used in another hour
power_balance = model.addConstrs((
                sum(Pgen[t,gen] for gen in range(N_gen))
              + sum(Pw[t,ren] for ren in range(N_ren))
              + Pdis[t]
              == 
                Pch[t]
              + sum(Pd[t,d] for d in range(N_d)) for t in range(T)
             ), name="Power_Balance")

#Adding the storage unit

# Storage charging constraints
model.addConstrs((Pch[t] >= 0
                for t in range(T)), name ="Charging Min Capacity")

model.addConstrs((Pch[t] <= P_ch
                for t in range(T)), name ="Charging Max Capacity")

# Storage discharging constraints
model.addConstrs((Pdis[t] >= 0
                for t in range(T)), name ="Discharging Min Capacity")

model.addConstrs((Pdis[t] <= P_dis
                for t in range(T)), name ="Discharging Max Capacity")

# Storage capacity constraints
model.addConstrs((e[t] >= 0
                  for t in range(T)), name ="Storage Min Capacity")

model.addConstrs((e[t] <= E
                  for t in range(T)), name ="Storage Max Capacity")

# Modelling charge and discharge
model.addConstrs((e[t] == e[t-1] + eta_ch * Pch[t] - (1/eta_dis) * Pdis[t] for t in range(1,T)), name="Storage Balance")

# Initial condition of the storage unit
model.addConstr(e[0] == 0.5 * E, name="Initial Capacity")


# Storage is not included in the objective function, described in the Note 2 of the assignment description
# It can be utilized, but does neither bid or offer on energy market

# Objective function
model.setObjective(sum(
                 sum(Pd[t,d] * df_utility.iloc[t,d] for d in range(N_d))
                 - sum(Pgen[t,gen] * df_gen.loc[gen,"Ci"] for gen in range(N_gen))
                 - sum(Pw[t,ren] * 0 for ren in range(N_ren)) for t in range(T)), sense=GRB.MAXIMIZE)

model.update()

# Solve the optimization problem
model.optimize()




Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (win64 - Windows 11+.0 (26200.2))

CPU model: AMD Ryzen 5 6600U with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 1872 rows, 912 columns and 2805 nonzeros
Model fingerprint: 0xc5f1b728
Coefficient statistics:
  Matrix range     [9e-01, 1e+00]
  Objective range  [1e+00, 5e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e+01, 6e+02]
Presolve removed 1825 rows and 787 columns
Presolve time: 0.01s
Presolved: 47 rows, 125 columns, 193 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.8879597e+06   1.052264e+03   0.000000e+00      0s
      54    1.8467959e+06   0.000000e+00   0.000000e+00      0s

Solved in 54 iterations and 0.02 seconds (0.00 work units)
Optimal objective  1.846795892e+06


#### Data processing and analysis

In [15]:
# Init
clear_price = []
total_operating_price = 0
total_profit_prod = []
profit_conv = []
profit_wind = []
consumer_surplus = []
social_welfare = 0
# Clearing price for each hour:
for t in range(T):
    clear_price.append(power_balance[t].Pi)


# Total Operating price:
P_gen_list = [[Pgen[t, gen].X for gen in range(N_gen)] for t in range(T)]
P_wind_list = [[Pw[t, ren].X for ren in range(N_ren)] for t in range(T)]
P_D_list = [[Pd[t, d].X for d in range(N_d)] for t in range(T)]

for t in range(T):
    for gen in range(N_gen):
        total_operating_price += -1 * (clear_price[t] * P_gen_list[t][gen])

# Total profit for each producer:
profit_conv = [[-1 * (clear_price[t] - df_gen.loc[gen, "Ci"]) * P_gen_list[t][gen] 
            for gen in range(N_gen)]
            for t in range(T)]    

profit_wind = [[-1 * (clear_price[t] - 0) * P_wind_list[t][ren] 
            for ren in range(N_ren)]
            for t in range(T)]    

consumer_surplus = [[(df_utility.iloc[t,d] - clear_price[t]) * P_D_list[t][d] 
            for d in range(N_d)]
            for t in range(T)] 

# Social welfare
# Sum of profits for all generators + sum of clearing price and bid price for demand
social_welfare = sum(sum(profit_conv,[])) + sum(sum(profit_wind,[])) + sum(sum(consumer_surplus,[]))
